In [3]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.wrappers.scikit_learn import KerasClassifier
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import SGD
from keras.constraints import maxnorm

Using TensorFlow backend.


In [4]:
# convert series to supervised learning & normalize input variables
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg


In [5]:
# Function to create model, required for KerasClassifier

#1. Batch Size and Epochs Model
def create_model_1():
    
    # create model
    model = Sequential()
    model.add(LSTM(12, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    return model

#2. Training Optimization Model
def create_model_2(optimizer='adam'):
    
    # create model
    model = Sequential()
    model.add(LSTM(12, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    model.compile(loss='mae', optimizer=optimizer, metrics=["accuracy"])
    
    return model

#3. Learning Rate and Momentum Model
def create_model_3(learn_rate=0.01, momentum=0):
    
    # create model
    model = Sequential()
    model.add(LSTM(128, input_shape=(train_X.shape[1], train_X.shape[2])))
    model.add(Dense(1))
    optimizer = SGD(lr=learn_rate, momentum=momentum)
    model.compile(loss='mae', optimizer=optimizer, metrics=["accuracy"])
    
    return model

#4. Network Weight Initialization Model
def create_model_4(init_mode='uniform'):
    
    # create model
    model = Sequential()
    model.add(LSTM(12, input_shape=(train_X.shape[1], train_X.shape[2]), kernel_initializer=init_mode))
    model.add(Dense(1, kernel_initializer=init_mode))
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    
    return model

#5. Neuron Activation Function Model
def create_model_5(activation='relu'):
    # create model
    model = Sequential()
    model.add(LSTM(12, input_shape=(train_X.shape[1], train_X.shape[2]))) 
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    
    return model

#6. Dropout Regularization Model
def create_model_6(dropout_rate=0.0, weight_constraint=0):
    # create model
    model = Sequential()
    model.add(LSTM(12, input_shape=(train_X.shape[1], train_X.shape[2]), kernel_constraint=maxnorm(weight_constraint))) 
    model.add(Dropout(dropout_rate))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
    
    return model

#7. Number of Neurons in the Hidden Layer
def create_model_7(neurons=1):
    # create model
    model = Sequential()
    model.add(LSTM(neurons, input_shape=(train_X.shape[1], train_X.shape[2]), kernel_constraint=maxnorm(4)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    # Compile model
    model.compile(loss='mae', optimizer='adam', metrics=["accuracy"])
                  
    return model

In [6]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
#dataset = numpy.loadtxt("./reports/pima-indians-diabetes.data.csv", delimiter=",")
dataset = read_csv('./reports/company_report_2.csv', header=0, index_col="time")
dataset = dataset[['volume_tests', 'month', 'is_weekend', 'season', 'number_test_types', 'numbers_tested']]
values = dataset.values

# specify the number of days and features 
n_days = 7
n_features = 6

# integer encode direction
encoder = LabelEncoder()
values[:,3] = encoder.fit_transform(values[:,3])
print(values)

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# split into input (X) and output (Y) variables
X = values[:,0:5]
Y = values[:,0]

# frame as supervised learning
reframed = series_to_supervised(scaled, n_days, 1)
print(reframed.shape)

[[21963 1 1 3 3 234]
 [38224 1 0 3 3 2071]
 [39066 1 0 3 3 2090]
 ...
 [20947 12 1 3 4 1318]
 [35616 12 0 3 4 4153]
 [36029 12 0 3 4 4157]]
(1088, 48)


In [7]:
# split into train and test sets
values = reframed.values

train_size = int(len(values) * 0.70)
test_size = len(values) - train_size
train, test = values[:train_size,:], values[train_size:,:]

# split into input and outputs
n_obs = n_days * n_features
train_X, train_y = train[:, :n_obs], train[:, -n_features]
test_X, test_y = test[:, :n_obs], test[:, -n_features]
print(train_X.shape, len(train_X), train_y.shape)

# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_days, n_features))
test_X = test_X.reshape((test_X.shape[0], n_days, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(761, 42) 761 (761,)
(761, 7, 6) (761,) (327, 7, 6) (327,)


## 1. Tune Batch Size and Number of Epochs

In [8]:
# create model
model = KerasClassifier(build_fn=create_model_1, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]# summarize results
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

/home/shanebowen/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [9]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.030581 using {'batch_size': 100, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 10, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 10, 'epochs': 50}
0.000000 (0.000000) with: {'batch_size': 10, 'epochs': 100}
0.003058 (0.004325) with: {'batch_size': 20, 'epochs': 10}
0.003058 (0.004325) with: {'batch_size': 20, 'epochs': 50}
0.000000 (0.000000) with: {'batch_size': 20, 'epochs': 100}
0.006116 (0.008650) with: {'batch_size': 40, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 40, 'epochs': 50}
0.003058 (0.004325) with: {'batch_size': 40, 'epochs': 100}
0.000000 (0.000000) with: {'batch_size': 60, 'epochs': 10}
0.003058 (0.004325) with: {'batch_size': 60, 'epochs': 50}
0.003058 (0.004325) with: {'batch_size': 60, 'epochs': 100}
0.006116 (0.008650) with: {'batch_size': 80, 'epochs': 10}
0.000000 (0.000000) with: {'batch_size': 80, 'epochs': 50}
0.000000 (0.000000) with: {'batch_size': 80, 'epochs': 100}
0.030581 (0.043248) with: {'batch_size': 100, 'epochs':

## 2. Tune Training Optimization Algorithm

In [9]:
# create model
model = KerasClassifier(build_fn=create_model_2, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

In [10]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.027461 using {'optimizer': 'SGD'}
0.027461 (0.029563) with: {'optimizer': 'SGD'}
0.004566 (0.006458) with: {'optimizer': 'RMSprop'}
0.000000 (0.000000) with: {'optimizer': 'Adagrad'}
0.000000 (0.000000) with: {'optimizer': 'Adadelta'}
0.000000 (0.000000) with: {'optimizer': 'Adam'}
0.004566 (0.006458) with: {'optimizer': 'Adamax'}
0.009259 (0.013095) with: {'optimizer': 'Nadam'}


## 3. Tune Learning Rate and Momentum

In [7]:
# create model
model = KerasClassifier(build_fn=create_model_3, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
param_grid = dict(learn_rate=learn_rate, momentum=momentum)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

/home/shanebowen/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [8]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.036530 using {'learn_rate': 0.001, 'momentum': 0.4}
0.000000 (0.000000) with: {'learn_rate': 0.001, 'momentum': 0.0}
0.000000 (0.000000) with: {'learn_rate': 0.001, 'momentum': 0.2}
0.036530 (0.051661) with: {'learn_rate': 0.001, 'momentum': 0.4}
0.018455 (0.017340) with: {'learn_rate': 0.001, 'momentum': 0.6}
0.000000 (0.000000) with: {'learn_rate': 0.001, 'momentum': 0.8}
0.004566 (0.006458) with: {'learn_rate': 0.001, 'momentum': 0.9}
0.009132 (0.012915) with: {'learn_rate': 0.01, 'momentum': 0.0}
0.000000 (0.000000) with: {'learn_rate': 0.01, 'momentum': 0.2}
0.000000 (0.000000) with: {'learn_rate': 0.01, 'momentum': 0.4}
0.018265 (0.025830) with: {'learn_rate': 0.01, 'momentum': 0.6}
0.000000 (0.000000) with: {'learn_rate': 0.01, 'momentum': 0.8}
0.018519 (0.026189) with: {'learn_rate': 0.01, 'momentum': 0.9}
0.018455 (0.017340) with: {'learn_rate': 0.1, 'momentum': 0.0}
0.032154 (0.016984) with: {'learn_rate': 0.1, 'momentum': 0.2}
0.013889 (0.011340) with: {'learn_rate':

## 4. Tune Network Weight Initialization

In [14]:
# create model
model = KerasClassifier(build_fn=create_model_4, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

In [15]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.004566 using {'init_mode': 'lecun_uniform'}
0.000000 (0.000000) with: {'init_mode': 'uniform'}
0.004566 (0.006458) with: {'init_mode': 'lecun_uniform'}
0.000000 (0.000000) with: {'init_mode': 'normal'}
0.000000 (0.000000) with: {'init_mode': 'zero'}
0.000000 (0.000000) with: {'init_mode': 'glorot_normal'}
0.000000 (0.000000) with: {'init_mode': 'glorot_uniform'}
0.000000 (0.000000) with: {'init_mode': 'he_normal'}
0.004566 (0.006458) with: {'init_mode': 'he_uniform'}


## 5. Tune Neuron Activation Function

In [16]:
# create model
model = KerasClassifier(build_fn=create_model_5, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

In [17]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.004566 using {'activation': 'softplus'}
0.000000 (0.000000) with: {'activation': 'softmax'}
0.004566 (0.006458) with: {'activation': 'softplus'}
0.000000 (0.000000) with: {'activation': 'softsign'}
0.000000 (0.000000) with: {'activation': 'relu'}
0.000000 (0.000000) with: {'activation': 'tanh'}
0.000000 (0.000000) with: {'activation': 'sigmoid'}
0.004566 (0.006458) with: {'activation': 'hard_sigmoid'}
0.000000 (0.000000) with: {'activation': 'linear'}


## 6. Tune Dropout Regularization

In [7]:
# create model
model = KerasClassifier(build_fn=create_model_6, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
param_grid = dict(dropout_rate=dropout_rate, weight_constraint=weight_constraint)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

/home/shanebowen/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










In [8]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.036530 using {'dropout_rate': 0.9, 'weight_constraint': 4}
0.000000 (0.000000) with: {'dropout_rate': 0.0, 'weight_constraint': 1}
0.000000 (0.000000) with: {'dropout_rate': 0.0, 'weight_constraint': 2}
0.000000 (0.000000) with: {'dropout_rate': 0.0, 'weight_constraint': 3}
0.004566 (0.006458) with: {'dropout_rate': 0.0, 'weight_constraint': 4}
0.000000 (0.000000) with: {'dropout_rate': 0.0, 'weight_constraint': 5}
0.000000 (0.000000) with: {'dropout_rate': 0.1, 'weight_constraint': 1}
0.000000 (0.000000) with: {'dropout_rate': 0.1, 'weight_constraint': 2}
0.004566 (0.006458) with: {'dropout_rate': 0.1, 'weight_constraint': 3}
0.000000 (0.000000) with: {'dropout_rate': 0.1, 'weight_constraint': 4}
0.000000 (0.000000) with: {'dropout_rate': 0.1, 'weight_constraint': 5}
0.000000 (0.000000) with: {'dropout_rate': 0.2, 'weight_constraint': 1}
0.004566 (0.006458) with: {'dropout_rate': 0.2, 'weight_constraint': 2}
0.000000 (0.000000) with: {'dropout_rate': 0.2, 'weight_constraint': 

## 7. Tune Number of Neurons in the Hidden Layer

In [9]:
# create model
model = KerasClassifier(build_fn=create_model_7, epochs=100, batch_size=10, verbose=0)
# define the grid search parameters
neurons = [1, 5, 10, 15, 20, 25, 30]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(test_X, test_y)

In [10]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.004566 using {'neurons': 15}
0.000000 (0.000000) with: {'neurons': 1}
0.000000 (0.000000) with: {'neurons': 5}
0.000000 (0.000000) with: {'neurons': 10}
0.004566 (0.006458) with: {'neurons': 15}
0.000000 (0.000000) with: {'neurons': 20}
0.000000 (0.000000) with: {'neurons': 25}
0.000000 (0.000000) with: {'neurons': 30}
